In [1]:
# Model design
import agentpy as ap
from random import choice

# Visualization
import matplotlib.pyplot as plt
import matplotlib.animation
import IPython
import numpy as np

from RoadAgent import RoadAgent
from City import City, Road
from City import Direction
from CarAgent import CarAgent
from TrafficLightAgent import TrafficLightAgent

In [2]:
class CityModel(ap.Model):
    @staticmethod
    def __get_positions(grid: list[list]) -> list[tuple[int, int]]:
        positions = []
        for i, column in enumerate(grid):
            for j, space in enumerate(column):
                if type(space) is Road:
                    positions.append((i, j))
        return positions

    def setup(self):
        # Unpack parameters
        self.city: City = self.p["city"]
        self.n_cars: int = self.p["cars"]
        self.spawn_points: list[tuple[int, int]] = self.p["spawn_points"]
        self.size: tuple[int, int] = self.p["size"]

        # Create grid (city)
        self.grid = ap.Grid(self, size, track_empty=True, torus=True)

        # Create agents
        roads = ap.AgentList(self, city.amount_of_roads, RoadAgent)
        road_positions = self.__get_positions(self.city.grid)
        self.grid.add_agents(roads, road_positions)
        for road in self.grid.agents:
            position = self.grid.positions[road]
            x, y = position
            road.add_directions(self.city.grid[x][y].directions)

        self.__add_traffic_lights()

        cars = ap.AgentList(self, self.n_cars, CarAgent)
        self.grid.add_agents(cars, positions=[choice(self.spawn_points) for _ in range(self.n_cars)])

    def step(self):
        cars_to_delete = ap.AgentList(self, cls=CarAgent)
        for agent in self.grid.agents:
            if agent.type == "CarAgent":
                direction = agent.select_next_direction()
                if agent.check_if_can_move(direction):
                    agent.move(direction)
                else:
                    cars_to_delete.append(agent)
            elif agent.type == "TrafficLightAgent":
                agent.check_status()
        self.grid.remove_agents(cars_to_delete)

    def update(self):
        pass

    def end(self):
        pass

    def __add_traffic_lights(self):
        traffic_lights = ap.AgentList(self, cls=TrafficLightAgent)
        for road in self.grid.agents:
            if len(road.directions) > 1:  # There is an intersection
                for direction in road.directions:
                    traffic_lights.append(TrafficLightAgent(self, direction))

## Single Run Animation

In [3]:
# Define parameters

# time in s

size = (10, 10)
city = City(size[0], size[1])
city.add_street((4, 0), (4, 10), Direction.RIGHT)
city.add_street((0, 4), (10, 4), Direction.UP)

parameters = {
    "size": size,
    "city": city,
    "cars": 3,
    "spawn_points": [(0, 4), (4, 0)],
    "steps": 20
}

In [5]:
# Create single-run animation with custom colors


def map_agent_to_number(agents: ap.AgentIter):
    agent: ap.Agent
    road_exists = False
    for agent in agents:
        if agent.type == 'TrafficLightAgent':
            return 2
        if agent.type == 'CarAgent':
            return 1
        if agent.type == 'RoadAgent':
            road_exists = True
    if road_exists:  # There is road and no cars
        return 0
    return np.nan  # There are no agents in that position, use numpy.nan instead of None


def animation_plot(model, ax):
    attr_grid = model.grid.apply(map_agent_to_number)  # apply(map_agent_to_number)
    color_dict = {0: '#7777', 1: '#d62c2c', 2: '#e5e5e5', None: '#d5e5d5'}
    ap.gridplot(attr_grid, color_dict, True, ax)
    ax.set_title(f"Simulation of a city\n"
                 f"Time-step: {model.t}")


fig, ax = plt.subplots()
city_model = CityModel(parameters)
animation = ap.animate(city_model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))

[[None, None, None, None, <City.Road object at 0x00000177E64A96D0>, None, None, None, None, None], [None, None, None, None, <City.Road object at 0x00000177E64B2820>, None, None, None, None, None], [None, None, None, None, <City.Road object at 0x00000177E64B2760>, None, None, None, None, None], [None, None, None, None, <City.Road object at 0x00000177E64B27F0>, None, None, None, None, None], [<City.Road object at 0x00000177E64A9700>, <City.Road object at 0x00000177E64A9A60>, <City.Road object at 0x00000177E64B23D0>, <City.Road object at 0x00000177E64B2520>, <City.Road object at 0x00000177E64B28B0>, <City.Road object at 0x00000177E64B2340>, <City.Road object at 0x00000177E64B2A90>, <City.Road object at 0x00000177E64B25E0>, <City.Road object at 0x00000177E64B2190>, <City.Road object at 0x00000177E64B2940>], [None, None, None, None, <City.Road object at 0x00000177E64B2790>, None, None, None, None, None], [None, None, None, None, <City.Road object at 0x00000177E64B24C0>, None, None, None, No